In [44]:
import pandas as pd
from scipy.sparse import lil_matrix
import scipy.spatial.distance

In [45]:
stats = pd.read_csv('Full_NFL_Stats_2022.csv')

In [46]:
stats.head()

,Unnamed: 0,teamName,displayName,description,abbreviation,value,perGameValue,rank
0,0,Falcons,Fumbles,The number of times a player/team has fumbled ...,FUM,21.0,1.0,13.0
1,1,Falcons,Fumbles Lost,The number of times a fumble is recovered by t...,LST,12.0,NaN,4.0
2,2,Falcons,Forced Fumbles,The total number of forced fumbles.,FF,8.0,NaN,62.0
3,3,Falcons,Fumbles Recovered,The number of fumbles recovered.,FR,7.0,NaN,23.0
4,4,Falcons,Fumbles Touchdowns,The number of times a fumbles is recovered and...,FTD,1.0,NaN,8.0


In [47]:
stats.drop(columns=['perGameValue', 'Unnamed: 0'],axis=1, inplace=True)

In [48]:
rec_rat = stats.loc[stats['displayName'] == 'ESPN Widereceiver Rating']
rec_rat = rec_rat.sort_values(by='rank')

In [49]:
rec_rat.tail()

,teamName,displayName,description,abbreviation,value,rank
8302,Ravens,ESPN Widereceiver Rating,The ESPN Widereceiver Rating.,ESPNWR,5367.0,28.0
2548,Titans,ESPN Widereceiver Rating,The ESPN Widereceiver Rating.,ESPNWR,5212.0,29.0
7754,Panthers,ESPN Widereceiver Rating,The ESPN Widereceiver Rating.,ESPNWR,5141.0,30.0
82,Falcons,ESPN Widereceiver Rating,The ESPN Widereceiver Rating.,ESPNWR,4807.0,31.0
630,Bears,ESPN Widereceiver Rating,The ESPN Widereceiver Rating.,ESPNWR,4378.0,32.0


In [50]:
receptions = stats.loc[stats['displayName'] == 'Receptions']
receptions = receptions.sort_values(by='rank')

In [51]:
receptions.tail()

,teamName,displayName,description,abbreviation,value,rank
8317,Ravens,Receptions,The total number of receptions.,REC,300.0,28.0
2563,Titans,Receptions,The total number of receptions.,REC,285.0,29.0
7769,Panthers,Receptions,The total number of receptions.,REC,267.0,30.0
97,Falcons,Receptions,The total number of receptions.,REC,257.0,31.0
645,Bears,Receptions,The total number of receptions.,REC,223.0,32.0


In [52]:
ypg = stats.loc[stats['displayName'] == 'Receiving Yards Per Game']
ypg = ypg.sort_values(by='rank')

In [53]:
ypg.tail()

,teamName,displayName,description,abbreviation,value,rank
7768,Panthers,Receiving Yards Per Game,The average number of receiving yards per game.,YDS/G,190.941,28.0
2562,Titans,Receiving Yards Per Game,The average number of receiving yards per game.,YDS/G,189.824,29.0
8316,Ravens,Receiving Yards Per Game,The average number of receiving yards per game.,YDS/G,188.353,30.0
96,Falcons,Receiving Yards Per Game,The average number of receiving yards per game.,YDS/G,172.176,31.0
644,Bears,Receiving Yards Per Game,The average number of receiving yards per game.,YDS/G,152.824,32.0


In [54]:
rec_td = stats.loc[stats['displayName'] == 'Receiving Touchdowns']
rec_td = rec_td.sort_values(by='value',ascending=False)
rec_td = rec_td.drop_duplicates(subset=['teamName'])

In [55]:
rec_td.tail()

,teamName,displayName,description,abbreviation,value,rank
3802,Rams,Receiving Touchdowns,The total number of receiving touchdowns.,REC,16.0,28.0
2706,Titans,Receiving Touchdowns,The total number of receiving touchdowns.,REC,16.0,28.0
7764,Panthers,Receiving Touchdowns,The total number of receiving touchdowns.,TD,16.0,28.0
5298,Jets,Receiving Touchdowns,The total number of receiving touchdowns.,TD,15.0,31.0
6268,Steelers,Receiving Touchdowns,The total number of receiving touchdowns.,REC,12.0,32.0


In [56]:
rec_big = stats.loc[stats['abbreviation'] == 'BIG']
rec_big = rec_big.sort_values(by='value',ascending=False)

In [57]:
rec_big

,teamName,displayName,description,abbreviation,value,rank
3101,Chiefs,20+ Yard Receiving Plays,The number of receiving big plays gained.,BIG,73.0,1.0
5567,Eagles,20+ Yard Receiving Plays,The number of receiving big plays gained.,BIG,63.0,2.0
3923,Dolphins,20+ Yard Receiving Plays,The number of receiving big plays gained.,BIG,62.0,3.0
3375,Raiders,20+ Yard Receiving Plays,The number of receiving big plays gained.,BIG,58.0,4.0
4471,Patriots,20+ Yard Receiving Plays,The number of receiving big plays gained.,BIG,57.0,5.0
...,...,...,...,...,...,...
881,Bengals,20+ Yard Rushing Plays,The number of rushing big plays gained.,BIG,7.0,26.0
8553,Texans,20+ Yard Rushing Plays,The number of rushing big plays gained.,BIG,7.0,26.0
6361,Chargers,20+ Yard Rushing Plays,The number of rushing big plays gained.,BIG,6.0,30.0
4169,Vikings,20+ Yard Rushing Plays,The number of rushing big plays gained.,BIG,5.0,31.0


In [58]:
bottom_wr = []

for df in [rec_rat, receptions, ypg, rec_td, rec_big]:

    bottom_names = list(df['teamName'].tail(16))

    bottom_wr += bottom_names

name_counts = {name: bottom_wr.count(name) for name in set(bottom_wr)}

final_names = [name for name in name_counts.keys() if name_counts[name] == 5]

print(final_names)

['Rams', 'Texans']


In [59]:
wrsim = pd.read_csv('WRData.csv')

In [60]:
wrsimdex_df = wrsim.set_index('PlayerName')

In [61]:
t_player = 'Justin Jefferson'
t_player_stat = wrsimdex_df.loc[t_player]
dist = scipy.spatial.distance.cdist(wrsimdex_df, [t_player_stat], metric = "cosine")[:,0]
query_distances = list(zip(wrsim.index, dist))

In [62]:
for sim_play, sim_stats in sorted(query_distances, key = lambda x: x[1], reverse = False)[:11]:
    print(wrsim['PlayerName'].iloc[sim_play], sim_stats)

Justin Jefferson 0.0
Terry McLaurin 0.00013372603519112847
Garrett Wilson 0.00020551501922572157
Mike Williams 0.0003440165112315041
Jaylen Waddle 0.0003669064700236868
A.J. Brown 0.00042413521287931744
Tyreek Hill 0.0005889352745903631
CeeDee Lamb 0.0006840447886427015
Christian Kirk 0.0009501619154335117
Darius Slayton 0.0009829837436252653
Jerry Jeudy 0.0009956640095383218
